# Mistral मॉडल्स के साथ निर्माण

## परिचय

इस पाठ में शामिल हैं:
- विभिन्न Mistral मॉडल्स का अन्वेषण
- हर मॉडल के उपयोग के मामले और परिस्थितियों को समझना
- कोड उदाहरण जो हर मॉडल की खासियतें दिखाते हैं


## मिस्ट्रील मॉडल्स

इस पाठ में, हम 3 अलग-अलग मिस्ट्रील मॉडल्स के बारे में जानेंगे:  
**मिस्ट्रील लार्ज**, **मिस्ट्रील स्मॉल** और **मिस्ट्रील नीमो**।

इनमें से हर एक मॉडल Github मॉडल मार्केटप्लेस पर मुफ्त में उपलब्ध है। इस नोटबुक में दिया गया कोड इन्हीं मॉडल्स का उपयोग करके कोड चलाएगा। Github मॉडल्स का उपयोग करके [AI मॉडल्स के साथ प्रोटोटाइपिंग](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) के बारे में और जानकारी यहां दी गई है।


## Mistral Large 2 (2407)
Mistral Large 2 फिलहाल Mistral का प्रमुख मॉडल है और इसे एंटरप्राइज उपयोग के लिए डिज़ाइन किया गया है।

यह मॉडल मूल Mistral Large का एक उन्नत संस्करण है, जिसमें ये सुधार किए गए हैं:
-  बड़ा कॉन्टेक्स्ट विंडो - 128k बनाम 32k
-  गणित और कोडिंग कार्यों में बेहतर प्रदर्शन - औसतन 76.9% सटीकता बनाम 60.4%
-  बहुभाषी प्रदर्शन में वृद्धि - इसमें ये भाषाएँ शामिल हैं: अंग्रेज़ी, फ्रेंच, जर्मन, स्पेनिश, इटालियन, पुर्तगाली, डच, रूसी, चीनी, जापानी, कोरियाई, अरबी और हिंदी।

इन खूबियों के साथ, Mistral Large इन क्षेत्रों में उत्कृष्ट है:
- *Retrieval Augmented Generation (RAG)* - बड़े कॉन्टेक्स्ट विंडो के कारण
- *Function Calling* - इस मॉडल में नेटिव फंक्शन कॉलिंग है, जिससे इसे बाहरी टूल्स और APIs के साथ जोड़ा जा सकता है। ये कॉल्स एक साथ या क्रम में एक के बाद एक की जा सकती हैं।
- *Code Generation* - यह मॉडल Python, Java, TypeScript और C++ कोड जनरेशन में बेहतरीन है।


इस उदाहरण में, हम Mistral Large 2 का उपयोग एक टेक्स्ट दस्तावेज़ पर RAG पैटर्न चलाने के लिए कर रहे हैं। सवाल कोरियाई भाषा में लिखा गया है और लेखक की कॉलेज से पहले की गतिविधियों के बारे में पूछता है।

यह Cohere Embeddings Model का उपयोग टेक्स्ट दस्तावेज़ और सवाल दोनों के एम्बेडिंग्स बनाने के लिए करता है। इस उदाहरण के लिए, यह faiss Python पैकेज को एक वेक्टर स्टोर के रूप में उपयोग करता है।

Mistral मॉडल को भेजे गए प्रॉम्प्ट में सवाल और वे प्राप्त किए गए हिस्से शामिल होते हैं जो सवाल से मिलते-जुलते हैं। इसके बाद मॉडल स्वाभाविक भाषा में उत्तर प्रदान करता है।


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## मिस्टरल स्मॉल
मिस्टरल स्मॉल, मिस्टरल मॉडल्स परिवार का एक और मॉडल है जो प्रीमियर/एंटरप्राइज श्रेणी के अंतर्गत आता है। जैसा कि नाम से स्पष्ट है, यह एक स्मॉल लैंग्वेज मॉडल (SLM) है। मिस्टरल स्मॉल का उपयोग करने के फायदे हैं:
- लागत में बचत, मिस्टरल LLMs जैसे मिस्टरल लार्ज और NeMo की तुलना में - 80% तक कीमत में कमी
- कम लेटेंसी - मिस्टरल के LLMs की तुलना में तेज़ प्रतिक्रिया
- लचीला - इसे विभिन्न वातावरणों में कम संसाधनों की आवश्यकता के साथ तैनात किया जा सकता है।

मिस्टरल स्मॉल इन कार्यों के लिए उपयुक्त है:
- टेक्स्ट आधारित कार्य जैसे सारांश बनाना, भावना विश्लेषण और अनुवाद
- ऐसे एप्लिकेशन जहाँ लागत प्रभावशीलता के कारण बार-बार अनुरोध किए जाते हैं
- कम लेटेंसी वाले कोड कार्य जैसे रिव्यू और कोड सुझाव


## मिस्टरल स्मॉल और मिस्टरल लार्ज की तुलना

मिस्टरल स्मॉल और लार्ज के बीच लेटेंसी में अंतर दिखाने के लिए, नीचे दिए गए सेल्स चलाएँ।

आपको रिस्पॉन्स टाइम में 3-5 सेकंड का अंतर दिखेगा। साथ ही, एक ही प्रॉम्प्ट पर रिस्पॉन्स की लंबाई और शैली पर भी ध्यान दें।


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## मिस्ट्रील नीमो

इस पाठ में चर्चा किए गए अन्य दो मॉडलों की तुलना में, मिस्ट्रील नीमो एकमात्र मुफ्त मॉडल है जो Apache2 लाइसेंस के साथ आता है।

इसे मिस्ट्रील के पहले ओपन सोर्स LLM, मिस्ट्रील 7B, का एक अपग्रेड माना जाता है।

नीमो मॉडल की कुछ अन्य विशेषताएँ हैं:

- *और अधिक कुशल टोकनाइज़ेशन:* यह मॉडल आमतौर पर इस्तेमाल होने वाले tiktoken की जगह Tekken टोकनाइज़र का उपयोग करता है। इससे यह अधिक भाषाओं और कोड पर बेहतर प्रदर्शन करता है।

- *फाइनट्यूनिंग:* बेस मॉडल फाइनट्यूनिंग के लिए उपलब्ध है। इससे उन उपयोग मामलों के लिए अधिक लचीलापन मिलता है जहाँ फाइनट्यूनिंग की आवश्यकता हो सकती है।

- *नेटिव फंक्शन कॉलिंग* - मिस्ट्रील लार्ज की तरह, इस मॉडल को फंक्शन कॉलिंग पर प्रशिक्षित किया गया है। यह इसे खास बनाता है क्योंकि यह ऐसा करने वाले पहले ओपन सोर्स मॉडलों में से एक है।


## मिस्ट्रील नीमो

इस पाठ में चर्चा किए गए अन्य दो मॉडलों की तुलना में, मिस्ट्रील नीमो एकमात्र मुफ्त मॉडल है जो Apache2 लाइसेंस के साथ आता है।

इसे मिस्ट्रील के पहले ओपन सोर्स LLM, मिस्ट्रील 7B का अपग्रेड माना जाता है।

नीमो मॉडल की कुछ अन्य विशेषताएँ हैं:

- *और अधिक कुशल टोकनाइज़ेशन:* यह मॉडल आमतौर पर इस्तेमाल होने वाले tiktoken की जगह Tekken टोकनाइज़र का उपयोग करता है। इससे यह अधिक भाषाओं और कोड पर बेहतर प्रदर्शन करता है।

- *फाइनट्यूनिंग:* बेस मॉडल फाइनट्यूनिंग के लिए उपलब्ध है। इससे उन उपयोग मामलों के लिए अधिक लचीलापन मिलता है जहाँ फाइनट्यूनिंग की आवश्यकता हो सकती है।

- *नेटिव फंक्शन कॉलिंग* - मिस्ट्रील लार्ज की तरह, इस मॉडल को फंक्शन कॉलिंग पर प्रशिक्षित किया गया है। यह इसे खास बनाता है क्योंकि यह ऐसा करने वाले पहले ओपन सोर्स मॉडलों में से एक है।


### टोकनाइज़र की तुलना

इस उदाहरण में, हम देखेंगे कि Mistral NeMo टोकनाइज़ेशन को Mistral Large की तुलना में कैसे संभालता है।

दोनों उदाहरणों में एक ही प्रॉम्प्ट लिया गया है, लेकिन आप देखेंगे कि NeMo, Mistral Large की तुलना में कम टोकन लौटाता है।


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## सीखना यहीं नहीं रुकता, अपनी यात्रा जारी रखें

इस पाठ को पूरा करने के बाद, हमारे [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) को देखें और अपनी जनरेटिव एआई की जानकारी को और बेहतर बनाएं!



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
